# Trénink s destilací nad datasetem TREC (coarse) s modelem BiLSTM
V tomto notebooku je trénován model BiLSTM s odemčenou embedding vrstvou nad původním i augmentovaným datasetem TREC (coarse), jako učitelský model je využíván finetunued BERT nad stejným datasetem.

Pro původní i augmentovaný dataset je na základě nalezených hyperparametrů ze sešitu hp_search proveden normální trénink a trénink s destilací znalostí. V rámci tréninků je oproti prohledávání hyperparametrů využito EarlyStoppingu pro zamezení přeučení. Navíc jsou získány také výsledky nad testovací částí datasetu a další metriky využívané v práci (velikost modelu a rychlost inference).

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits. Konfigurace jednotlivých tréninků odpovídá výstup pěti nejlepších běhů z prohledávání hyperparametrů u dané konfigurace. Maximální délka tréninku je nastavena na 20 epoch. EarlyStopping pracuje s trpělivostí čtyř epoch.

## Import knihoven a základní nastavení

In [1]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
from torch.utils.data import DataLoader
import kagglehub
import torch
import base
import os
import copy

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Resetování náhodného seedu pro replikovatelnost výsledků.

In [ ]:
base.reset_seed()

Ověření dostupnosti GPU.

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [2]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_coarse")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_coarse")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_coarse")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_coarse", f"~/data/{DATASET}/test-logits_coarse", f"~/data/{DATASET}/train-logits-augmented_coarse"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Tokenizace.

In [6]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [7]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [8]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [9]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [10]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Část tokenů nebyla nalezena, což ovšem nepředstavuje problém.

In [11]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


Přiřazení indexu tokenům v každé části datasetu.

In [12]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

Zarovnání délky všech záznamů.

In [13]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

Přidání ID tokenů do každé části datasetu.

In [14]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

Příprava dataloaderů pro finální ověření rychlosti inference. Testování probíhá pouze nad jedním záznamem z trénovací části.

In [15]:
train_data_gpu = copy.deepcopy(train_data)
train_data_gpu.set_format(type="torch", columns=["input_ids"], device="cuda")
gpu_data_loader = DataLoader(train_data_gpu, batch_size=1, shuffle=False)

train_data_cpu = copy.deepcopy(train_data)
train_data_cpu.set_format(type="torch", columns=["input_ids"], device="cpu")
cpu_data_loader = DataLoader(train_data_cpu, batch_size=1, shuffle=False)

## Normální trénink s původním datasetem

Získání modelu s definovanou embedding vrstvou.

In [107]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=False)

In [108]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(8768, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=6, bias=True)
)


Konfigurace tréninku, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [109]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_coarse_embedd", logging_dir=f"~/logs/{DATASET}/bilstm-base_coarse_embedd", lr=.0042, weight_decay=.002, warmup_steps=2, epochs=20)

In [110]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 4 epoch. 

In [111]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [112]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.019500,0.709314,0.762603,0.678191,0.646255,0.650726
2,0.413200,0.550586,0.825848,0.801200,0.747617,0.750404
3,0.165200,0.467583,0.871677,0.865483,0.830585,0.845452
4,0.050800,0.580546,0.865261,0.863276,0.808262,0.828051
5,0.011400,0.724800,0.873511,0.840925,0.826007,0.831467
6,0.002600,0.783802,0.877177,0.853414,0.827879,0.838675
7,0.001000,0.813848,0.876260,0.856967,0.837371,0.845690
8,0.001100,0.807218,0.871677,0.850550,0.833145,0.840461
9,0.001900,0.901794,0.860678,0.851119,0.813781,0.829042
10,0.004900,0.721986,0.878093,0.860112,0.848954,0.853798


TrainOutput(global_step=490, training_loss=0.11945776052307337, metrics={'train_runtime': 61.7332, 'train_samples_per_second': 1412.853, 'train_steps_per_second': 11.339, 'total_flos': 0.0, 'train_loss': 0.11945776052307337, 'epoch': 14.0})

Přepnutí modelu do evaluačního režimu.


In [113]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(8768, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=6, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [114]:
trainer.evaluate(test_data)

{'eval_loss': 0.5840951204299927,
 'eval_accuracy': 0.906,
 'eval_precision': 0.8639466785055578,
 'eval_recall': 0.904462318152026,
 'eval_f1': 0.8781455979538942,
 'eval_runtime': 2.9781,
 'eval_samples_per_second': 167.892,
 'eval_steps_per_second': 1.343,
 'epoch': 14.0}

Uložení modelu.


In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-embedd_coarse.pth")

## Trénink s destilací s původním datasetem

Získání studentského modelu s definovanou embedding vrstvou.

In [ ]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=False)

Konfigurace tréninku s destilací, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_coarse", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_coarse", lr=.0045, weight_decay=0.004, warmup_steps=1, epochs=20, lambda_param=.6, temp=4)

In [ ]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 4 epoch. 

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.579400,1.505238,0.778185,0.660902,0.667053,0.658325
2,0.860500,0.909297,0.868928,0.732875,0.737967,0.734576
3,0.327400,0.870254,0.867094,0.865468,0.789401,0.813618
4,0.178800,0.802691,0.877177,0.884460,0.820388,0.841270
5,0.109000,0.748609,0.881760,0.890055,0.814244,0.837671
6,0.085600,0.716064,0.882676,0.891276,0.823415,0.846143
7,0.074600,0.724488,0.886343,0.894990,0.836061,0.857259
8,0.066600,0.708397,0.882676,0.889546,0.824575,0.846037
9,0.059300,0.704430,0.886343,0.893422,0.826708,0.848988
10,0.056400,0.711276,0.879010,0.890723,0.839644,0.859264


TrainOutput(global_step=595, training_loss=0.278290374138776, metrics={'train_runtime': 71.7215, 'train_samples_per_second': 1216.093, 'train_steps_per_second': 9.76, 'total_flos': 0.0, 'train_loss': 0.278290374138776, 'epoch': 17.0})

Přepnutí studenta do evaluačního režimu.

In [ ]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(8768, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=6, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.5296463966369629,
 'eval_accuracy': 0.932,
 'eval_precision': 0.919648381583738,
 'eval_recall': 0.9098615870573079,
 'eval_f1': 0.9138968580144518,
 'eval_runtime': 3.2689,
 'eval_samples_per_second': 152.959,
 'eval_steps_per_second': 1.224,
 'epoch': 17.0}

Uložení studentského modelu.

In [32]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-embedd_coarse.pth")

## Normální trénink s augmentovaným datasetem

Získání modelu s definovanou embedding vrstvou.

In [67]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=False)

Konfigurace tréninku, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.


In [68]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd-aug_coarse", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd-aug_coarse", lr=.002, weight_decay=.008, warmup_steps=5, epochs=20)

In [69]:
base.reset_seed()

Konfigurace trenéra s trpělivostí 4 epoch. 

In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)


Spuštění tréninku, výstupy nad validační částí datasetu.

In [71]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.359100,0.529261,0.863428,0.870381,0.811015,0.829143
2,0.046300,0.552197,0.880843,0.850891,0.850367,0.850046
3,0.019000,0.675151,0.874427,0.875528,0.845152,0.857035
4,0.011300,0.831786,0.873511,0.876119,0.833242,0.850206
5,0.008100,0.791679,0.866178,0.861656,0.835946,0.847474
6,0.007200,0.772654,0.870761,0.854103,0.843329,0.847732
7,0.004400,0.896939,0.873511,0.871044,0.852020,0.859693
8,0.003400,0.994705,0.870761,0.853022,0.852182,0.850521
9,0.002000,1.014341,0.877177,0.867550,0.847385,0.856057
10,0.001500,1.190608,0.872594,0.865412,0.843736,0.852923


TrainOutput(global_step=3344, training_loss=0.0421322101023066, metrics={'train_runtime': 80.7977, 'train_samples_per_second': 9622.794, 'train_steps_per_second': 75.25, 'total_flos': 0.0, 'train_loss': 0.0421322101023066, 'epoch': 11.0})

Přepnutí modelu do evaluačního režimu.


In [72]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(8768, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=6, bias=True)
)


Otestování modelu nad testovací částí datasetu.

In [73]:
trainer.evaluate(test_data)

{'eval_loss': 0.5354726910591125,
 'eval_accuracy': 0.918,
 'eval_precision': 0.9033578164814812,
 'eval_recall': 0.9113238366157065,
 'eval_f1': 0.9039567889040537,
 'eval_runtime': 3.2855,
 'eval_samples_per_second': 152.183,
 'eval_steps_per_second': 1.217,
 'epoch': 11.0}

Uložení modelu.


In [74]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-embedd-aug_coarse.pth")

## Trénink s destilací s augmentovaným datasetem

Získání studentského modelu s definovanou embedding vrstvou.

In [75]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=6, freeze_embed=False)

Konfigurace tréninku s destilací, zvolené parametry odpovídají pěti nejlepším výstupům z prohledávání hyperparametrů.

In [76]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd-aug_coarse", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd-aug_coarse", lr=.001,  epochs=20, weight_decay=.008, warmup_steps=20, lambda_param=.8, temp=5.5)

In [77]:
base.reset_seed()

Konfigurace destilačního trenéra s trpělivostí 4 epoch. 

In [78]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)

Spuštění tréninku s destilací, výstupy nad validační částí datasetu.

In [79]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.369300,0.886910,0.882676,0.865977,0.806203,0.823402
2,0.249400,0.694973,0.901008,0.906112,0.867113,0.882553
3,0.157800,0.692235,0.900092,0.905024,0.865872,0.881404
4,0.123000,0.702153,0.898258,0.891591,0.865440,0.875792
5,0.103900,0.649190,0.898258,0.902928,0.855666,0.873377
6,0.092600,0.646503,0.902841,0.896042,0.869141,0.879681


TrainOutput(global_step=1824, training_loss=0.3493421611032988, metrics={'train_runtime': 43.8463, 'train_samples_per_second': 17732.379, 'train_steps_per_second': 138.666, 'total_flos': 0.0, 'train_loss': 0.3493421611032988, 'epoch': 6.0})

Přepnutí studenta do evaluačního režimu.

In [80]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(8768, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=6, bias=True)
)

Otestování studenta nad testovací částí datasetu.

In [81]:
trainer.evaluate(test_data)

{'eval_loss': 0.5639212727546692,
 'eval_accuracy': 0.942,
 'eval_precision': 0.94881746291823,
 'eval_recall': 0.9157969171439074,
 'eval_f1': 0.9289803585481407,
 'eval_runtime': 3.0934,
 'eval_samples_per_second': 161.633,
 'eval_steps_per_second': 1.293,
 'epoch': 6.0}

Uložení studentského modelu.

In [82]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-embedd-aug_coarse.pth")

Získání počtu trénovatelných parametrů v modelu. 

In [50]:
base.count_parameters(student_model)

model size: 16.472MB.
Total Trainable Params: 4318006.


,Modules,Parameters
0,embedding.weight,2630400
1,lstm.weight_ih_l0,360000
2,lstm.weight_hh_l0,360000
3,lstm.bias_ih_l0,1200
4,lstm.bias_hh_l0,1200
5,lstm.weight_ih_l0_reverse,360000
6,lstm.weight_hh_l0_reverse,360000
7,lstm.bias_ih_l0_reverse,1200
8,lstm.bias_hh_l0_reverse,1200
9,fc1.weight,240000


Změření rychlosti inference při použití CPU, 1000 pokusů s jedním záznamem.

In [51]:
cpu_benchmark = base.BenchMarkRunner(student_model, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  3.30 ms
  1 measurement, 1000 runs , 4 threads


Změření rychlosti inference při použití GPU, 1000 pokusů s jedním záznamem.

In [52]:
gpu_benchmark = base.BenchMarkRunner(student_model, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  1.73 ms
  1 measurement, 1000 runs , 4 threads
